In [1]:
import numpy as np
import matplotlib.pyplot as plt
import inspect
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import os

In [2]:
import importlib
import sys
sys.path.append('../')
from Utils.perturbationsnew import RBPIndividual, RBPIndividualNew1, RBPIndividualNew1fast, RBPIndividualNew2, zeroPerturb, noisePerturb, blurPerturb,multisamplebackgroundIdentification
from Utils.models import *
from Utils.data import perturb, generateSynthetic, generateSyntheticPerturbReal,ReadTS

#print(sys.path)


## Test important content removal (accuracy decrease)

### Load synthetic dataset and train CNN model 

In [3]:
x_train, y_train,x_test_original, y_test_original = ReadTS('Computers')
y_train = y_train - 1
y_test_original = y_test_original - 1
#print(x_train.shape)
#print(y_train)

In [4]:
from sklearn.preprocessing import StandardScaler
def normalize(TS):

    original_shape = TS.shape
    
    TS_2d = TS.reshape(original_shape[0] , original_shape[1])
    
    scaler = StandardScaler()
    
    TS_normalized = scaler.fit_transform(TS_2d)
    
    TS = TS_normalized.reshape(original_shape)
    
    return TS

x_train = normalize(x_train)
x_test_original = normalize(x_test_original)
#print(x_train[1])

In [5]:
model = make_CNN_model(x_train.shape[1:],num_classes = 2)
trained_model, history = train_CNN_model(model, x_train, y_train, epochs=100, batch_size=32)

Epoch 1/100


2024-12-03 00:12:39.160017: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


7/7 [==============================] - 1s 44ms/step - loss: 0.6703 - sparse_categorical_accuracy: 0.5850 - val_loss: 0.7438 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 2/100
7/7 [==============================] - 0s 28ms/step - loss: 0.5601 - sparse_categorical_accuracy: 0.6550 - val_loss: 0.7600 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 3/100
7/7 [==============================] - 0s 27ms/step - loss: 0.5338 - sparse_categorical_accuracy: 0.7400 - val_loss: 0.7638 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 4/100
7/7 [==============================] - 0s 29ms/step - loss: 0.5139 - sparse_categorical_accuracy: 0.7800 - val_loss: 0.7284 - val_sparse_categorical_accuracy: 0.1200
Epoch 5/100
7/7 [==============================] - 0s 30ms/step - loss: 0.4982 - sparse_categorical_accuracy: 0.7800 - val_loss: 0.6988 - val_sparse_categorical_accuracy: 0.4600
Epoch 6/100
7/7 [==============================] - 0s 29ms/step - loss: 0.4817 - sparse_categorical_accuracy: 

In [6]:
test_loss, test_acc = trained_model.evaluate(x_test_original, y_test_original)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

8/8 [==============================] - 0s 8ms/step - loss: 0.9149 - sparse_categorical_accuracy: 0.7640
Test accuracy 0.7639999985694885
Test loss 0.9148957133293152


In [7]:
def l2norm(perturb):
    difference = perturb - x_test_original
    
    squared_difference = np.square(difference)
    
    sum_of_squares = np.sum(squared_difference, axis=1)
    
    l2_norms = np.sqrt(sum_of_squares)
    
    average_l2_norm = np.mean(l2_norms)
    
    #print(average_l2_norm)

    return average_l2_norm


def perturb_predict(strategy,index_0,index_1):
    if strategy =='RBP1fast':
        bg = multisamplebackgroundIdentification(perturb_test_set)
        x_test_pert = np.asarray([perturb(strategy,x,index_0,index_1,bg = bg) for x in perturb_test_set])
        x_test_pert = np.expand_dims(x_test_pert, axis=-1)
        predictions_pert = model.predict(x_test_pert)
        predictions_pert = np.argmax(predictions_pert,axis=1)
    else:
        x_test_pert = np.asarray([perturb(strategy,x,index_0,index_1) for x in perturb_test_set])
        x_test_pert = np.expand_dims(x_test_pert, axis=-1)
        predictions_pert = model.predict(x_test_pert)
        predictions_pert = np.argmax(predictions_pert,axis=1)

    accuracy = accuracy_score(y_test_original, predictions_pert)
    precision = precision_score(y_test_original, predictions_pert, average='weighted')  
    recall = recall_score(y_test_original, predictions_pert, average='weighted')
    f1 = f1_score(y_test_original, predictions_pert, average='weighted')
    l2 = l2norm(x_test_pert)

    # print(strategy)
    # print(accuracy)
    # print(precision)
    # print(recall)
    # print(f1)

    return accuracy,precision,recall,f1,l2

In [9]:
perturb_test_set = x_test_original.copy()
perturb_test_set = perturb_test_set.reshape(x_test_original.shape[0:2])
index_0 = 0
index_1 = 200

strategies = ['RBP', 'RBP1fast', 'zero', 'noise', 'blur']

results = []
l2_results = []

for strategy in strategies:
    accuracy, precision, recall, f1, l2 = perturb_predict(strategy, index_0, index_1)
    results.append([strategy, accuracy, precision, recall, f1])
    l2_results.append(l2)

columns = ['Strategy', 'Accuracy', 'Precision', 'Recall', 'F1 Score']
df_results = pd.DataFrame(results, columns=columns)
l2_columns = ['L2-Norm']
l2_results = pd.DataFrame(l2_results, columns=l2_columns)

predictions_original = model.predict(x_test_original)
predictions_original = np.argmax(predictions_original,axis=1)
accuracy = accuracy_score(y_test_original, predictions_original)
precision = precision_score(y_test_original, predictions_original, average='weighted')  
recall = recall_score(y_test_original, predictions_original, average='weighted')
f1 = f1_score(y_test_original, predictions_original, average='weighted')
original_data = ['original', accuracy, precision, recall, f1]
df_results.loc[-1] = original_data  
df_results.index = df_results.index + 1  
df_results = df_results.sort_index()  

df_results.loc[2, 'Strategy'] = 'MRBP'

current_directory = os.getcwd()
result_folder = os.path.join(current_directory, 'result_data')

csv_path = os.path.join(result_folder, 'Computers.csv')
df_results.to_csv(csv_path, index=False)

csv_path = os.path.join(result_folder, 'Computersl2.csv')
l2_results.to_csv(csv_path, index=False)

print("done")
print(df_results)
print(l2_results)

8/8 [==============================] - 0s 6ms/step
done
   Strategy  Accuracy  Precision  Recall  F1 Score
0  original     0.764   0.776920   0.764  0.761215
1       RBP     0.740   0.742624   0.740  0.739295
2      MRBP     0.536   0.593423   0.536  0.451754
3      zero     0.724   0.724129   0.724  0.723960
4     noise     0.552   0.668954   0.552  0.458246
5      blur     0.740   0.745676   0.740  0.738490
     L2-Norm
0   8.023789
1  10.853251
2  10.280979
3  36.742094
4   2.600488
